In [1]:
import igraph
from igraph import *
igraph.__version__

'0.7.1'

In [2]:
import cairo
import matplotlib
%matplotlib inline

In [3]:
num_vertices = 18
tree = Graph()
tree.add_vertices(num_vertices)
tree.add_edges([(0,1), (0,2), (1,3), (1,4), (2,5), (2,6), (2,7),
        (3,8), (3,9), (4,10), (4,11), (4,12), (11,13), (11,14),
        (14,15), (14,16), (14,17)])
tree.vs["description"] = ["cohort total", "no graduation date",
        "graduation date", "no withdrawal reason", "withdrawal reason",
        "4 year graduation", "5 year graduation", "more than 5 years",
        "no withdrawal date", "district withdrawal date", "misc withdrawal",
        "transferred", "dropout", "no withdraw to IRN", "withdrawn to IRN",
        "dropout recovery program", "JVSD/career tech", "other Ohio IRN"]
assert(len(tree.vs["description"]) == num_vertices)
tree.vs["count"] = [0] * num_vertices
tree.vs["students"] = [None] * num_vertices

In [4]:
outcome_buckets = {}
outcome_buckets["non-terminal"] = ["cohort total", "no graduation date", "graduation date", 
                "no withdrawal reason", "withdrawal reason", "transferred", "withdrawn to IRN"]
outcome_buckets["exclude"] = ["misc withdrawal"]
outcome_buckets["dropout"] = ["dropout", "dropout recovery program"]
outcome_buckets["uncertain"] = ["no withdrawal date", "district withdrawal date", 
                               "no withdraw to IRN", "JVSD/career tech", "other Ohio IRN"]
outcome_buckets["late"] = ["5 year graduation", "more than 5 years"]
outcome_buckets["on-time"] = ["4 year graduation"]
outcome_buckets_flipped = dict((outcome, 
                                [k for k,v in outcome_buckets.items() if outcome in v][0]) 
                               for outcome in tree.vs["description"])
tree.vs["outcomes"] = [outcome_buckets_flipped[bucket] for bucket in tree.vs["description"]]

In [5]:
print(tree) # 18 vertices, 17 edges, 4 vertex attributes, 0 edge attributes

IGRAPH U--- 18 17 --
+ attr: count (v), description (v), outcomes (v), students (v)
+ edges:
0--1 0--2 1--3 1--4 2--5 2--6 2--7 3--8 3--9 4--10 4--11 4--12 11--13 11--14
14--15 14--16 14--17


In [6]:
len(tree.get_edgelist())

17

In [7]:
cohort_vertex = tree.vs.select(description='cohort total')
assert(len(cohort_vertex) == 1)
cohort_index = [v.index for v in cohort_vertex][0]
tree.vs[cohort_index].attributes()

{'count': 0,
 'description': 'cohort total',
 'outcomes': 'non-terminal',
 'students': None}

In [8]:
assert([v["count"] for v in cohort_vertex][0] == 0)
student_list = [1000, 2000, 3000, 4000]
### index first, then attribute name! reverse order doesn't change value
tree.vs[cohort_index]["students"] = student_list
tree.vs[cohort_index]["count"] = len(student_list)

In [9]:
assert([v["count"] for v in cohort_vertex][0] == len(student_list))
assert([v["students"] for v in cohort_vertex][0] == student_list)
tree.vs[cohort_index].attributes()

{'count': 4,
 'description': 'cohort total',
 'outcomes': 'non-terminal',
 'students': [1000, 2000, 3000, 4000]}

In [21]:
tree_layout = tree.layout_reingold_tilford(root=[0])

In [22]:
color_dict = {"non-terminal":"black", "exclude":"yellow", "dropout":"red", "uncertain":"green", "late":"blue", 
             "on-time":"magenta"}
visual_style = {}
visual_style["vertex_size"] = 30
visual_style["vertex_color"] = [color_dict[category] for category in tree.vs["outcomes"]]
visual_style["vertex_label"] = ["{desc}\n{count}".format(desc=desc, count=count) 
                                for desc,count in zip(tree.vs["description"], tree.vs["count"])]
visual_style["layout"] = tree_layout
visual_style["vertex_label_dist"] = 2
visual_style["bbox"] = (800, 800)
visual_style["margin"] = 60
visual_style["vertex_label_size"] = 10
try:
    plot(tree, "test_tree_plot.png", **visual_style)
except TypeError:
    print("ignore TypeError")